In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns

warnings.simplefilter(action="ignore", category=FutureWarning)
sns.set_theme(color_codes=True, style="whitegrid")

In [2]:
data = pd.read_csv("characteristics_data_feb2017.csv")

data.head()

,Unnamed: 0,yy,mm,date,permno,ret,q10,q20,q50,prc,...,beta,cum_return_12_2,cum_return_12_7,cum_return_1_0,cum_return_36_13,idio_vol,spread_mean,suv,rel_to_high_price,lev
0,1,1996,7,1996-07-31,10002,-0.031532,98261.1000,204261.900,813573.5625,-13.4375,...,0.166116,0.086772,0.089053,0.007207,0.110756,0.028370,0.090090,-0.457116,0.925000,0.123299
1,2,1996,8,1996-08-31,10002,0.004651,93952.7000,187060.500,751569.1875,-13.5000,...,0.164960,0.094605,0.050158,-0.031532,0.110756,0.022565,0.083100,-0.341980,0.895833,0.123299
2,3,1996,9,1996-09-30,10002,0.016667,96730.5625,195165.475,775633.8125,-13.6250,...,0.136445,0.060091,0.079329,0.004651,0.077763,0.022960,0.053797,-0.328501,0.900000,0.123299
3,4,1996,10,1996-10-31,10002,-0.018349,97944.0000,203921.000,795242.2500,13.3750,...,0.111756,-0.050393,0.012650,0.016667,0.173885,0.018144,0.082009,-0.217109,0.908333,0.123299
4,5,1996,11,1996-11-30,10002,0.000000,95892.6000,204784.050,785273.0000,-13.3750,...,0.109087,0.119904,0.124042,-0.018349,-0.029107,0.016831,0.091743,9.564113,0.922414,0.123299
